In [3]:
from __future__ import print_function
import keras
import tempfile
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import CSVLogger  #, UpdatePruningStep
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_callbacks
import tensorflow_model_optimization as tfmot
import numpy as np
import time
import os
import pickle
from keras.datasets import cifar100
import tensorflow as tf

In [ ]:
#@title
import multiprocessing
import os
import random
#import portpicker
#import hypertune
import tensorflow as tf

In [2]:
!pip install tensorflow-model-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 238 kB 6.3 MB/s 


In [4]:
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [ ]:
strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")  #   tf.distribute.experimental.CentralStorageStrategy()   #tf.distribute.MirroredStrategy()



2022-12-16 21:40:40.680265: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 21:40:41.407450: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 21:40:41.407768: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 21:40:41.602168: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [ ]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [ ]:
def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [ ]:
# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------

In [ ]:
def tflite_conv(model,path,quant=False):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    if quant == True:
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
    pruned_tflite_model = converter.convert()
    with open(path, 'wb') as f:
      f.write(pruned_tflite_model)
    print('Saved pruned TFLite model to:',path)

In [ ]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
    import os
    import zipfile

    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(file)

    return os.path.getsize(zipped_file)

In [ ]:
strategy.num_replicas_in_sync

1

In [ ]:
def resnet_training(x_train,y_train,x_test, y_test,layers,frequency,\
                    initial_sparsity,final_sparsity,gpu="T4",\
                    const=True, poly=False,file_name='cifar10_test1',\
                    num_classes=10,begin_step=0,end_step='default'):

    # Default parameters
    batch_size = 128 * strategy.num_replicas_in_sync 
    epochs = 2#300
    data_augmentation = True
    n = layers

    # Computed depth from supplied model parameter n
    depth = n * 6 + 2

    # Input image dimensions.
    input_shape = x_train.shape[1:]

    # Subtracting pixel mean improves accuracy
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    print('y_train shape:', y_train.shape)

    ####Changes start#####
    num_images = x_train.shape[0] #* (1 - validation_split)

    if end_step == 'default':
        end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

    #hyperparameters: initial_sparsity=0.50, final_sparsity=0.80
    if poly:
        pruning_params = {
              'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
                                                                       final_sparsity=final_sparsity,
                                                                       begin_step=begin_step,
                                                                       end_step=end_step,
                                                                      frequency=frequency)
        }
    if const:
            pruning_params = {
              'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=final_sparsity,
                                                                       begin_step=begin_step,
                                                                       end_step=end_step,
                                                                       frequency=frequency)
        }

    prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
    with strategy.scope():
        model = resnet_v1(input_shape=input_shape, depth=depth,num_classes=num_classes)
        model = prune_low_magnitude(model, **pruning_params)    #_for_pruning

        model.compile(loss= 'categorical_crossentropy',     #''  tf.  keras.losses.SparseCategoricalCrossentropy(from_logits=True)
                  optimizer=Adam(lr=lr_schedule(0)),
                  metrics=['accuracy'])
    ####Changes end#####
    model.summary()

    # Prepare model model saving directory.
    save_dir = os.path.join(os.getcwd(), 'saved_models')
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)

    model_run=file_name+'_'+str(depth)+'_'+gpu
    model_path=model_run+'.h5'
    print("RRrrrrrR",model_path)
    
    filepath = os.path.join(save_dir, model_path)
    print("RRR",filepath)
    
    print(model_run)

    # Prepare callbacks for model saving and for learning rate adjustment.
    checkpoint = ModelCheckpoint(filepath=filepath,
                                 monitor='val_acc',
                                 verbose=1,
                                 save_best_only=True)

    lr_scheduler = LearningRateScheduler(lr_schedule)

    lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                                   cooldown=0,
                                   patience=5,
                                   min_lr=0.5e-6)
    logdir = tempfile.mkdtemp()
    time_callback = TimeHistory()
    logname='/home/jupyter/final_proj/log_'+model_run+'.csv'
    csv_logger = CSVLogger(logname, append=True, separator=';')
    callbacks = [checkpoint, lr_reducer, lr_scheduler, csv_logger, time_callback, pruning_callbacks.UpdatePruningStep(), tfmot.sparsity.keras.PruningSummaries(log_dir=logdir)]

    st = time.time()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
    #%tensorboard --logdir={logdir}
    training_time = time.time() - st

    print(f"overall training time is {training_time}")
    epoch_times = time_callback.times
    print(f"each epoch training time is {epoch_times}")

    # Score trained model.
    scores = model.evaluate(x_test, y_test, verbose=1)
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])
    print("tt",save_dir+'/'+model_run)
    
    #save standard model 
    model.save(save_dir+'/'+model_path)
    
    #saving data 
    with open(model_run+'.pickle', 'wb') as handle:
        pickle.dump([training_time,epoch_times,scores[0],scores[1]], handle)

    #save pruned model
    model_for_export = tfmot.sparsity.keras.strip_pruning(model)
    pruned_keras_file = save_dir+'/PRUNE_'+model_path
    keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
    print('Saved pruned Keras model to:', pruned_keras_file)
    
    #convert to tflite+ save
    pruned_tflite_file=save_dir+'/lite_PRUNE_'+model_path
    tflite_conv(model,pruned_tflite_file)
    
    #save to tf lite + qaunt
    pruned_tflite_quant_file=save_dir+'/lite_quant_PRUNE_'+model_path
    tflite_conv(model,pruned_tflite_quant_file,quant=True)
    
    pruned=get_gzipped_model_size(pruned_keras_file)
    tflite_pruned=get_gzipped_model_size(pruned_tflite_file)
    tflite_quant_pruned=get_gzipped_model_size(pruned_tflite_quant_file)
    
    
    print("Size of gzipped pruned Keras model: %.2f bytes" % (pruned))
    print("Size of gzipped pruned TFlite model: %.2f bytes" % (tflite_pruned))
    print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (tflite_quant_pruned))
    
    with open(model_run+'.pickle', 'wb') as handle:
        pickle.dump([training_time,epoch_times,scores[0],scores[1],pruned,tflite_pruned,tflite_quant_pruned], handle)

    return(model,model_path)

In [ ]:
# Load the CIFAR100 data.
(X_train, Y_train), (X_test, Y_test) = cifar100.load_data()
# Normalize data.
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
# Convert class vectors to binary class matrices.
Y_train = keras.utils.to_categorical(Y_train)
Y_test = keras.utils.to_categorical(Y_test)

# Experimenting with constant sparsity 
In order to see the effects of pruning on a network with a very large number of parameters,Resnet44, verses in network with fewer parameters like,Resnet20, we first investigated the effects of constant sparsity versus polynomials varsity scheduling.

Resnet44

In [ ]:
sparsity_level=[0.6,0.8,0.9]

names=["sixty","eighty","ninety"]
models=[]
#for x in sparsity_level:
for x in range(2,3):
    name="resnet44_poly_sparsity_"+names[x]
    print(name)
    model1 = resnet_training(X_train,Y_train,X_test, Y_test,layers=7,frequency=100,\
                    initial_sparsity = 0.5, final_sparsity=sparsity_level[x],gpu="V100",\
                    const=False, poly=True,file_name=name,\
                    num_classes=100,begin_step=0,end_step='default')
    models=models+[model1]
    

resnet44_poly_sparsity_ninety
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 100)
Learning rate:  0.001
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 prune_low_magnitude_conv2d_135  (None, 32, 32, 16)  882         ['input_4[0][0]']                
  (PruneLowMagnitude)                                                                             
                                                                                                  
 prune_low_magnitude_batch_norm  (None, 32, 32, 16)  65          ['prune_low_magnitude_conv2d_135[
 alization_129 (PruneLowMagnitu       

/opt/conda/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


 alization_143 (PruneLowMagnitu                                  0][0]']                          
 de)                                                                                              
                                                                                                  
 prune_low_magnitude_add_69 (Pr  (None, 32, 32, 16)  1           ['prune_low_magnitude_activation_
 uneLowMagnitude)                                                141[0][0]',                      
                                                                  'prune_low_magnitude_batch_norma
                                                                 lization_143[0][0]']             
                                                                                                  
 prune_low_magnitude_activation  (None, 32, 32, 16)  1           ['prune_low_magnitude_add_69[0][0
 _143 (PruneLowMagnitude)                                        ]']                              
          

391/391 [==============================] - ETA: 0s - loss: 4.6498 - accuracy: 0.0509WARNING:tensorflow:Can save best model only with val_acc available, skipping.


391/391 [==============================] - 84s 85ms/step - loss: 4.6498 - accuracy: 0.0509 - val_loss: 4.6568 - val_accuracy: 0.0338 - lr: 0.0010
Learning rate:  0.001
Epoch 2/2
391/391 [==============================] - ETA: 0s - loss: 4.1193 - accuracy: 0.0976WARNING:tensorflow:Can save best model only with val_acc available, skipping.


...

INFO:tensorflow:Assets written to: /tmp/tmp16pt3bmv/assets
2022-12-16 04:18:44.497839: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-16 04:18:44.497910: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-16 04:18:44.498155: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp16pt3bmv
2022-12-16 04:18:44.666736: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-16 04:18:44.666802: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp16pt3bmv
2022-12-16 04:18:45.436012: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-16 04:18:47.907221: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmp16pt3bmv
2022-12-16 04:18:48.788761: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

Saved pruned TFLite model to: /home/jupyter/final_proj/saved_models/lite_PRUNE_resnet44_poly_sparsity_ninety_44_V100.h5


INFO:tensorflow:Assets written to: /tmp/tmpryutufkg/assets


INFO:tensorflow:Assets written to: /tmp/tmpryutufkg/assets
2022-12-16 04:21:36.014098: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-16 04:21:36.014179: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-16 04:21:36.014414: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpryutufkg
2022-12-16 04:21:36.186117: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-16 04:21:36.186173: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpryutufkg
2022-12-16 04:21:36.948706: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-16 04:21:39.421257: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmpryutufkg
2022-12-16 04:21:40.277025: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

Saved pruned TFLite model to: /home/jupyter/final_proj/saved_models/lite_quant_PRUNE_resnet44_poly_sparsity_ninety_44_V100.h5
Size of gzipped pruned Keras model: 575436.00 bytes
Size of gzipped pruned TFlite model: 637241.00 bytes
Size of gzipped pruned and quantized TFlite model: 637280.00 bytes


In [ ]:
sparsity_level=[0.6,0.8,0.9]
names=["sixty","eighty","ninety"]
models=[]
#for x in sparsity_level:
for x in range(3):
    name="resnet44_const_sparsity_"+names[x]
    print(name)
    model1 = resnet_training(X_train,Y_train,X_test, Y_test,layers=7,frequency=100,\
                    initial_sparsity = False, final_sparsity=sparsity_level[x],gpu="T4",\
                    const=True, poly=False,file_name=name,\
                    num_classes=100,begin_step=0,end_step='default')
    models=models+[model1]
    

In [ ]:
model1

NameError: name 'model1' is not defined

In [ ]:
#####scrap########

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision

### Scrap

In [ ]:
model1 = resnet_training(X_train,Y_train,X_test, Y_test,layers=3,frequency=100,\
                    initial_sparsity = False, final_sparsity=0.3,gpu="T4",\
                    const=True, poly=False,file_name='cifar7_test1',\
                    num_classes=100,begin_step=0,end_step='default')

In [ ]:
%load_ext tensorboard
model1 = resnet_training(X_train,Y_train,X_test, Y_test,layers=3,frequency=100,\
                    initial_sparsity = False, final_sparsity=0.3,gpu="T4",\
                    const=True, poly=False,file_name='cifar7_test1',\
                    num_classes=100,begin_step=0,end_step='default')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 100)
Learning rate:  0.001
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 prune_low_magnitude_conv2d_42   (None, 32, 32, 16)  882         ['input_3[0][0]']                
 (PruneLowMagnitude)                                                                              
                                                                                                  
 prune_low_magnitude_batch_norm  (None, 32, 32, 16)  65          ['prune_low_magnitude_conv2d_42[0
 alization_38 (PruneLowMagnitud                                  ][0

overall training time is 35.96639800071716
each epoch training time is [31.73249888420105]
313/313 [==============================] - 2s 6ms/step - loss: 3.9833 - accuracy: 0.1183
Test loss: 3.9832944869995117
Test accuracy: 0.11829999834299088
tt /home/kal2257/saved_models/cifar7_test1_20_T4
Saved pruned Keras model to: /home/kal2257/saved_models/PRUNE_cifar7_test1_20_T4.h5


INFO:tensorflow:Assets written to: /tmp/tmp2thzh_v7/assets


INFO:tensorflow:Assets written to: /tmp/tmp2thzh_v7/assets
2022-12-10 23:27:25.752954: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-10 23:27:25.753007: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-10 23:27:25.754056: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp2thzh_v7
2022-12-10 23:27:25.811024: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-10 23:27:25.811079: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp2thzh_v7
2022-12-10 23:27:26.010131: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-12-10 23:27:26.079075: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-10 23:27:26.865426: I tensorflow/cc/saved_model/loader.cc:213] Running initializatio

Saved pruned TFLite model to: /home/kal2257/saved_models/lite_PRUNE_cifar7_test1_20_T4.h5


INFO:tensorflow:Assets written to: /tmp/tmpwvclnqeh/assets


INFO:tensorflow:Assets written to: /tmp/tmpwvclnqeh/assets
2022-12-10 23:28:21.566610: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-10 23:28:21.566668: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-10 23:28:21.566852: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpwvclnqeh
2022-12-10 23:28:21.623263: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-10 23:28:21.623318: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpwvclnqeh
2022-12-10 23:28:21.889817: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-10 23:28:22.682462: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmpwvclnqeh
2022-12-10 23:28:22.967156: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

Saved pruned TFLite model to: /home/kal2257/saved_models/lite_quant_PRUNE_cifar7_test1_20_T4.h5
Size of gzipped pruned Keras model: 839343.00 bytes
Size of gzipped pruned TFlite model: 892934.00 bytes
Size of gzipped pruned and quantized TFlite model: 892955.00 bytes


In [ ]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Pruned and quantized TFLite test_accuracy:', test_accuracy)
print('Pruned TF test accuracy:', model_for_pruning_accuracy)

In [ ]:
path = os.getcwd()
path

'/home/jupyter/final_proj'

In [ ]:
with open('/home/jupyter/final_proj/cifar7_test1_20_T4.pickle', 'rb') as handle:
    b1 = pickle.load(handle)
b1

[35.96639800071716,
 [31.73249888420105],
 3.9832944869995117,
 0.11829999834299088,
 839343,
 892934,
 892955]

In [ ]:
model3 = keras.models.load_model('/home/kal2257/saved_models/PRUNE_cifar5_test1_20_T4.h5')


In [ ]:
model2.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_63 (Conv2D)             (None, 32, 32, 16)   448         ['input_4[0][0]']                
                                                                                                  
 batch_normalization_57 (BatchN  (None, 32, 32, 16)  64          ['conv2d_63[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_57 (Activation)     (None, 32, 32, 16)   0           ['batch_normalization_57[0]

In [ ]:
model3.get_weights()

[array([[[[ 1.31791592e-01,  5.80994114e-02,  3.31603527e-01,
            7.68987387e-02, -3.04641783e-01, -2.74092122e-03,
           -2.17599317e-01, -1.81094527e-01, -2.20986053e-01,
           -5.89315712e-01, -4.91172522e-01, -2.80829556e-02,
           -1.94892138e-01,  2.27247462e-01,  2.34928757e-01,
            1.73476785e-01],
          [-7.09220171e-02, -3.89235258e-01,  7.98407942e-02,
           -8.37986916e-03,  8.43527764e-02,  9.90333259e-02,
            1.78135782e-02,  3.74973230e-02,  3.60301673e-01,
            2.88770646e-01, -3.37435335e-01, -7.85104409e-02,
            2.62750328e-01,  2.92895108e-01, -5.04738986e-02,
            7.19242617e-02],
          [-2.73054689e-01,  3.28764826e-01,  4.08030450e-01,
           -2.07959801e-01,  3.77648622e-01,  2.56940186e-01,
           -4.75658506e-01, -1.39780447e-01, -9.34910998e-02,
           -1.73302621e-01, -1.96264178e-01, -3.23021486e-02,
            3.66190404e-01,  9.89206582e-02,  2.14506701e-01,
            

In [ ]:
%load_ext tensorboard

In [ ]:
#docs_infra: no_execute
%tensorboard --logdir={logdir}

In [ ]:
pickle.dump([training_time,epoch_times,scores[0],scores[1],pruned,tflite_pruned,tflite_quant_pruned], handle)
